In [1]:
from parser.datareader   import DataReader
from parser.datasaver    import save_data
from parser.node         import Node

from transforms3d.euler import euler2mat,mat2euler
import cv2
import numpy as np
import math
import copy
import os
from pprint import pprint

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
def save_cam(dirpath,idx,cam_parameter):
    cam_path = dirpath + "/cam" + str(idx)
    if not os.path.isdir(cam_path):
        os.makedirs(cam_path)
    parameter_path = cam_path + "/" + "cam_parameter.npy"
    np.save(parameter_path,cam_parameter)
    return cam_path

In [3]:
def get_rvec_tvec(pos,angle):
    angle = np.deg2rad(angle)
    rmat  = euler2mat(angle[0],angle[1],angle[2],'rxyz')
    rvec = np.array(mat2euler(rmat))
    pos  = np.array(pos)
    tvec = -np.matmul(rmat, pos)
    return rvec,tvec

In [4]:
def get_cam_lst(focus,radius):
    pos_lst = []
    angle_lst = []
    
    #camera 0 right
    pos   = [0.0, 2, 10.0]
    angle = [0,0,0]
    pos_lst.append(pos)
    angle_lst.append(angle)
    
    #camera 1 right
    pos = [15.0, 3,  0.0]
    angle=[0,-90,0]
    pos_lst.append(pos)
    angle_lst.append(angle)
    
    #camera 2  right
    pos = [0.0,  2, -10.0]
    angle=[0,0,180]      
    pos_lst.append(pos)
    angle_lst.append(angle)
    
    #camera 3   right
    pos = [-15.0, 2,  0.0]
    angle=[0,90,0] 
    pos_lst.append(pos)
    angle_lst.append(angle)
    
    #camera 4 right
    pos  = [10.0, 2,  10.0]
    angle= [0,-45,0] 
    pos_lst.append(pos)
    angle_lst.append(angle)
    
    #camera 5 right
    pos  = [-10.0, 2,  -10.0]
    angle= [0,45,180]
    #angle= [0,-45,180] no-see
    pos_lst.append(pos)
    angle_lst.append(angle)
    
    return pos_lst,angle_lst

In [5]:
def get_extrinsics(rvec,tvec):
    extMat = np.zeros((3,4))
    R = euler2mat(rvec[0],rvec[1],rvec[2],'rxyz')
    extMat[:3,:3] = R
    extMat[:3,3] = tvec
    return extMat

In [6]:
def get_cam_parameter(idx,pos,angle):
    width = 640
    height = 480
    cam = {}
    rvec, tvec = get_rvec_tvec(pos,angle)
    intrinsics = np.zeros((3,3),dtype = np.float64)
    focal_len = 500
    intrinsics[0][0] = focal_len
    intrinsics[1][1] = focal_len
    intrinsics[0][2] = width/2
    intrinsics[1][2] = height/2
    intrinsics[2][2] = 1
    distortion = np.zeros((1, 5),dtype = np.float64)
    cam['id']   = 'cam_' + str(idx)
    cam['rvec'] = rvec
    cam['tvec'] = tvec
    cam['intrinsics'] = intrinsics
    cam['distortion'] = distortion
    cam['extrinsics'] = get_extrinsics(rvec,tvec)
    cam['res_w'] = width
    cam['res_h'] = height
    cam['focal_len'] = 500
    return cam

In [7]:
def make_2d_joints_lst(joints_lst_3d,rvec, tvec, intrinsics, distortion):
    joints_2d_lst = []
    for joints_3d in joints_lst_3d:
        joints_2d = {}
        for (idx,joint_3d) in joints_3d.items():
            # make tuple to np.array and reshape
            #print(idx,joint.pos)
            joint_2d = Node(joint_3d.name,joint_3d.index,joint_3d.parent_index)
            joint_2d.pos = np.array(joint_3d.pos).reshape(1,3)
            # compute 2d pos
            joint_2d.pos,_ = cv2.projectPoints(joint_2d.pos, rvec, tvec, intrinsics, distortion)
            joint_2d.pos = joint_2d.pos[0][0]
            joints_2d[idx] = joint_2d
        for joint in joints_2d.values():
            if(joint.parent_index != -1):
                joint.parent = joints_2d[joint.parent_index]
            else:
                joint.parent = None
        joints_2d_lst.append(joints_2d)
    return joints_2d_lst

In [8]:
def make_joints_square():
    nodes_num = 17
    joints = []
    joint = Node("point_0",nodes_num,nodes_num + 1)
    joint.pos = np.array([-10,15,-27], dtype = np.float64)
    joints.append(copy.deepcopy(joint))
    
    joint = Node("point_1",nodes_num + 1,nodes_num + 2)
    joint.pos = np.array([10,15,-27], dtype = np.float64)
    joints.append(copy.deepcopy(joint))
    
    joint = Node("point_2",nodes_num + 2,nodes_num + 3)
    joint.pos = np.array([10,15,-7], dtype = np.float64)
    joints.append(copy.deepcopy(joint))
    
    joint = Node("point_3",nodes_num + 3,nodes_num)
    joint.pos = np.array([-10,15,-7], dtype = np.float64)
    joints.append(copy.deepcopy(joint))

    return joints

In [9]:
def add_joints(joints,joints_square):
    max_index = len(joints)
    for vpoint in joints_square:
        joints[max_index] = copy.deepcopy(vpoint)
        max_index += 1
    for joint in joints.values():
        if(joint.parent_index == -1):
            joint.parent = None
        else:
            joint.parent = joints[joint.parent_index]

In [10]:
def add_square_to_lst(joints_lst):
    joints_square = make_joints_square()
    for joints in joints_lst:
        add_joints(joints,joints_square)

In [11]:
json_path_3dpoints = './Output/human_points_3d.json'

In [12]:
reader = DataReader(json_path_3dpoints)
reader.load_data()
print(reader.frames_num)
print(reader.joints_num)
joints_lst = reader.frames

4510
17


In [13]:
focus = [0,0,0]
cam_pos_lst,angle_lst = get_cam_lst(focus,radius = 50)

In [14]:
# for idx in range(len(cam_pos_lst)):
#     pos = cam_pos_lst[idx]
#     angle = angle_lst[idx]
#     cam_parameter = get_cam_parameter(idx,pos,angle)
#     save_cam("./Output/",idx,cam_parameter)
#     rvec = cam_parameter['rvec']
#     tvec = cam_parameter['tvec']
#     intrinsics = cam_parameter['intrinsics']
#     distortion = cam_parameter['distortion']
#     joints_lst_2d = make_2d_joints_lst(joints_lst,rvec, tvec, intrinsics, distortion)
#     output_json_path = './Output/human_points_2d_cam_' +  str(idx) + '.json'
#     output_npy_path = './Output/human_points_2d_cam_' + str(idx) + '.npy'
#     save_data(joints_lst_2d,output_json_path,output_npy_path,dimension=2)

In [15]:
for human_idx in range(1,30):
    for action_idx in range(1,30):
        dir_path = "./output_human/" + str(human_idx).zfill(2) + "/" + str(action_idx).zfill(2)
        json_3dpoints = dir_path + "/points_3d.json"
        if not os.path.isfile(json_3dpoints):
            continue
        reader = DataReader(json_3dpoints)
        reader.load_data()
        joints_lst = reader.frames
        for cam_idx in range(len(cam_pos_lst)):
            pos = cam_pos_lst[cam_idx]
            angle = angle_lst[cam_idx]
            cam_parameter = get_cam_parameter(cam_idx,pos,angle)
            cam_path = save_cam(dir_path,cam_idx,cam_parameter)
            rvec = cam_parameter['rvec']
            tvec = cam_parameter['tvec']
            intrinsics = cam_parameter['intrinsics']
            distortion = cam_parameter['distortion']
            joints_lst_2d = make_2d_joints_lst(joints_lst,rvec, tvec, intrinsics, distortion)
            output_json_path = cam_path + '/points_2d' + '.json'
            output_npy_path  = cam_path + '/points_2d' + '.npy'
            save_data(joints_lst_2d,output_json_path,output_npy_path,dimension=2)

In [16]:
print("Work Done!")

Work Done!
